### In this Note book we clustered the banking clients data based on banking Behaviour, the workbook workflow is as follows: 

* Load Data 
* Processing Data (Dummy variables, dropping nulls & columns & merging data from all three CSV files, demographics, accounts and transactions) 
* Variable selections using variance and covariance 
* create two separate customer segmentations (using clustering) to split them into 3-5 clusters by running KMeans with 3 clusters and grouping data by cluster
* visualize the created clusters using radar charts and compare them agains each other


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from numpy import linalg as LA
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from yellowbrick.cluster import KElbowVisualizer
from sklearn import metrics
%matplotlib inline

In [3]:
demographics = pd.read_csv('twm_customer.csv', delimiter = ';')
accounts = pd.read_csv('twm_accounts.csv', delimiter = ';')
transactions = pd.read_csv('twm_transactions.csv', delimiter = ';')

In [4]:
demographics.head(5)

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,name_prefix,first_name,last_name,street_nbr,street_name,postal_code,city_name,state_code
0,1362691,26150,46,5,1,M,2,,Donald ...,Marek ...,8298,Second ...,89194,Las Vegas,NV
1,1362487,6605,71,1,0,M,2,,ChingDyi ...,Moussavi ...,10603,Daffodil ...,90159,Los Angeles,CA
2,1363160,18548,38,8,0,F,1,,Rosa ...,Johnston ...,8817,Figueroa ...,90024,Los Angeles,CA
3,1362752,47668,54,3,0,F,1,,Lisa ...,Martin ...,676,Humble ...,90172,Los Angeles,CA
4,1362548,44554,59,9,2,F,4,,Barbara ...,O'Malley ...,6578,C ...,10138,New York City,NY


In [4]:
demographics['gender'] = demographics['gender'].str.rstrip()
demographics = pd.get_dummies(demographics, columns=['gender'])
demographics = demographics.select_dtypes(exclude=['object'])
#dropping unwanted numeric groups
demographics.drop(columns=['street_nbr', 'postal_code', 'gender_M'], inplace=True)
#drop missing values
demographics.dropna(inplace=True)
demographics.head(5)

,cust_id,income,age,years_with_bank,nbr_children,marital_status,gender_F
0,1362691,26150,46,5,1,2,0
1,1362487,6605,71,1,0,2,0
2,1363160,18548,38,8,0,1,1
3,1362752,47668,54,3,0,1,1
4,1362548,44554,59,9,2,4,1


In [5]:
accounts.head(5)

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,13628063,1362806,SV,Y,10.12.1995,NaN,1430.22,284.58
1,4561143213627090,1362709,CC,Y,15.3.1993,NaN,266.34,496.15
2,4561143213628360,1362836,CC,Y,18.3.1992,NaN,55.90,1000.00
3,13633112,1363311,CK,Y,6.7.1995,NaN,11017.13,968.46
4,4561143213633610,1363361,CC,Y,17.6.1994,NaN,849.37,462.28


In [6]:
accounts['acct_type'] = accounts['acct_type'].str.rstrip()
accounts['account_active'] = accounts['account_active'].str.rstrip()
accounts = pd.get_dummies(accounts, columns=['acct_type', 'account_active'])
accounts.drop(columns=['account_active_N', 'acct_end_date', 'acct_start_date'], inplace=True)
accounts.dropna(inplace=True)
accounts.head(5)

,acct_nbr,cust_id,starting_balance,ending_balance,acct_type_CC,acct_type_CK,acct_type_SV,account_active_Y
0,13628063,1362806,1430.22,284.58,0,0,1,1
1,4561143213627090,1362709,266.34,496.15,1,0,0,1
2,4561143213628360,1362836,55.90,1000.00,1,0,0,1
3,13633112,1363311,11017.13,968.46,0,1,0,1
4,4561143213633610,1363361,849.37,462.28,1,0,0,1


In [7]:
transactions.head(5)

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,27,13625623,0.00,0.00,0.0,3753.34,21.10.1995,121656,A,IQ
1,97,13628392,0.00,0.00,0.0,254.49,5.2.1995,153053,V,IQ
2,21,13630842,-97.57,-97.57,0.0,3819.56,23.7.1995,0,P,WD
3,44,13631412,-0.15,-0.15,0.0,224.05,30.1.1995,0,,FK
4,31,13625722,0.00,0.00,0.0,240.55,25.1.1995,204521,B,IQ


In [8]:
print(transactions['interest_amt'].value_counts()[0])
print(transactions['tran_amt'].value_counts()[0])
print(transactions['principal_amt'].value_counts()[0])
print(transactions['tran_time'].value_counts()[0])
print(transactions[transactions['tran_amt'] == transactions['principal_amt']].shape)
transactions['channel'] = transactions['channel'].str.rstrip()
transactions['tran_code'] = transactions['tran_code'].str.rstrip()
transactions = pd.get_dummies(transactions, columns=['channel', 'tran_code'])
#transactions = pd.get_dummies(transactions, columns=['tran_id'])
transactions.drop(columns=['interest_amt', 'principal_amt', 'tran_date', 'channel_', 'channel_A', 'tran_code_IQ'], inplace=True)
transactions.dropna(inplace=True)
transactions.head(5)

67543
10887
17522
23564
(67543, 10)


,tran_id,acct_nbr,tran_amt,new_balance,tran_time,channel_B,channel_C,channel_E,channel_H,channel_K,...,tran_code_DP,tran_code_FA,tran_code_FK,tran_code_FM,tran_code_FP,tran_code_FU,tran_code_IN,tran_code_PM,tran_code_TR,tran_code_WD
0,27,13625623,0.00,3753.34,121656,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,97,13628392,0.00,254.49,153053,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21,13630842,-97.57,3819.56,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,44,13631412,-0.15,224.05,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,31,13625722,0.00,240.55,204521,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
acts_trans = accounts.join(transactions.set_index('acct_nbr'), on='acct_nbr')
combined = acts_trans.join(demographics.set_index('cust_id'), on='cust_id')
# drop starting balance and ending balance here because that information is already contained in the transaction data
combined.drop(columns=['starting_balance', 'ending_balance', 'acct_nbr', 'cust_id'], inplace=True)
combined.head(5)

,acct_type_CC,acct_type_CK,acct_type_SV,account_active_Y,tran_id,tran_amt,new_balance,tran_time,channel_B,channel_C,...,tran_code_IN,tran_code_PM,tran_code_TR,tran_code_WD,income,age,years_with_bank,nbr_children,marital_status,gender_F
0,0,0,1,1,39.0,0.00,283.87,110747.0,0.0,0.0,...,0.0,0.0,0.0,0.0,39382,24,3,1,3,0
0,0,0,1,1,37.0,72.82,199.71,132930.0,0.0,0.0,...,0.0,0.0,0.0,0.0,39382,24,3,1,3,0
0,0,0,1,1,35.0,-186.89,186.89,121640.0,0.0,1.0,...,0.0,0.0,1.0,0.0,39382,24,3,1,3,0
0,0,0,1,1,33.0,-80.00,618.17,184545.0,0.0,0.0,...,0.0,0.0,0.0,1.0,39382,24,3,1,3,0
0,0,0,1,1,31.0,0.00,201.82,142217.0,1.0,0.0,...,0.0,0.0,0.0,0.0,39382,24,3,1,3,0


In [10]:
transactions.drop(columns=['acct_nbr'], inplace=True)
accounts.drop(columns=['acct_nbr', 'cust_id'], inplace=True)
demographics.drop(columns=['cust_id'], inplace=True)

In [ ]:
scaler = StandardScaler()
cols = list(combined.columns)
cols_scaled = [s + "_scaled" for s in cols]
combined_scaled = pd.DataFrame()
combined_scaled[cols_scaled] = scaler.fit_transform(combined[cols])

In [ ]:
#Cutting out columns with too-low variances
vt = VarianceThreshold(0.1)
combined_transformed = vt.fit_transform(combined_scaled)
selected_columns = combined_scaled.columns[vt.get_support()]
combined_transformed = pd.DataFrame(combined_transformed, columns = selected_columns)
combined_transformed.dropna(inplace=True)

#Dropping columns with too high a coviarance
combined_corr = combined_transformed.corr().abs()
indices = np.where(combined_corr > 0.5) 
indices = [(combined_corr.index[x], combined_corr.columns[y]) 
for x, y in zip(*indices) if x != y and x < y]

for idx in indices:
    try:
        combined_transformed.drop(idx[1], axis = 1, inplace=True)
    except KeyError:
        pass

In [ ]:
combined_transformed

In [ ]:
#Running KMeans with 3 clusters and grouping data by cluster
model = KMeans(4)
kmeans = model.fit(combined_transformed)
labels = kmeans.labels_
mod = model.predict(combined_transformed)
combined_transformed['cluster'] = mod
result = combined_transformed.groupby(['cluster']).mean()

In [ ]:
fig = go.Figure()

i=0
while i < 4:
  fig.add_trace(go.Scatterpolar(
    r = result.loc[i].values,
    theta = result.columns,
    name = f'cluster #{i}'
    ))
  i += 1

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range = [-1.2, 4.0]
    ),
  ),
  showlegend=True
)

fig.show()

In [ ]:
#sns.pairplot(combined_transformed, vars=['acct_type_CC_scaled', 'acct_type_SV_scaled', 'tran_code_DP_scaled', 'tran_code_IN_scaled', 'tran_code_TR_scaled', 'tran_amt_scaled', 'tran_time_scaled'], hue='cluster', diag_kind='hist')

In [ ]:
pca = PCA(n_components=2)
x_pca = pca.fit_transform(combined_transformed)
x_pca.shape

In [ ]:
sns.set(rc = {'figure.figsize':(15,15)})
sns.set_theme(style='darkgrid')
sns.scatterplot(x = x_pca[:,0], y = x_pca[:, 1], hue=labels, legend='full', palette="Set1")

In [ ]:
sns.scatterplot(x=combined_transformed['tran_time_scaled'], y=combined_transformed['tran_amt_scaled'], hue=labels, palette="Set1")

In [ ]:
sns.scatterplot(x=combined_transformed['acct_type_CC_scaled'], y=combined_transformed['tran_amt_scaled'], hue=labels, palette="Set1")

In [ ]:
#visualizer = KElbowVisualizer(model, k=(1,10))
#visualizer.fit(combined_transformed)
#visualizer.poof()

In [ ]:
j = 1
fig_loop, axes = plt.subplots(nrows=5, ncols=4, figsize=(15,15), sharex = True, sharey = True)
fig_loop.tight_layout()
while j < 21:
  combined_loop = combined_transformed.copy()
  model = KMeans(j)
  kmeans = model.fit(combined_loop)
  labels = kmeans.labels_
  mod = model.predict(combined_loop)
  combined_loop['cluster_loop'] = mod
  result = combined_loop.groupby(['cluster_loop']).mean()
  pca = PCA(n_components=2)
  x_pca = pca.fit_transform(combined_loop)
  sns.scatterplot(x=x_pca[:,0], y=x_pca[:, 1], hue=labels, legend = False, palette="Set1", ax = axes[(j-1) % 5, (j-1) // 5])
  j += 1